In [ ]:
from models.ABrain import Airfoil_Brain
import numpy as np
import matplotlib.pyplot as plt
import torch

# Initialize
current_ada_iter=65
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
airfoil_brain=Airfoil_Brain(model_path_header='./model_save/pp_model_save/ada{:d}'.format(current_ada_iter), device=device) # load prior model
airfoil_brain.std_var='uniform'

# methods
## 1) evaluate: evaluate the aerodynamic coefficients of the given airfoil geometries and flow conditions
##      - input: latent_norm or latent_org (norm = True or False), batch x 13 features (10 design variables + 3 flow conditions - Ma/AoA/Re_base)
##      - output: airfoil, c81_mean, c81_std, c81_std_decompose

## 2) geom_save: save geometry in pointwise dataformat to genearte mesh
##      - input: latent_norm or latent_org (norm = True or False), save_folder_path (default is geom_save folder)
##      - output: None (geometry data are saved in save_folder_path)

## 3) find_target_airfoil: find target airfoil, upper/lower surface (in non-decreasing order) of airfoil should be given
##      - input: upper and lower surface of airfoil cooridinate
#       - output: fitted_latent_org, fitted_airfoil, optimization_results

In [ ]:
# example of evaluate() function

latent_geom=np.load('fitted_latent_naca23012.npy') # load geometry 

# flow condition
Ma=0.7 
AoA=2.3
Re_base=6e6

flow_con=np.array([[Ma, AoA, Re_base]])

latent=np.concatenate([latent_geom, flow_con],axis=1) # batch x features formatting
print(latent.shape)

airfoils, c81_mean, c81_std, c81_std_decompose = airfoil_brain.evaluate(latent, norm=True)
c81_mean[:,1]=10**c81_mean[:,1] # log transformation of drag coefficient

In [ ]:
# evaluate() use case: prediction of c81 table

# Define the grids
Ma_grid=np.array([0.2, 0.3, 0.4, 0.5, 0.6, 0.65, 0.7, 0.75, 0.8])
AoA_grid=np.array([-15, -13, -11, -9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25])
Re_base_grid = np.array([5e6, 5e6])

# Create meshgrid
Ma, AoA, Re_base = np.meshgrid(Ma_grid, AoA_grid, Re_base_grid, indexing='ij')

# Flatten the grids
Ma_flat = Ma.flatten()
AoA_flat = AoA.flatten()
Re_base_flat = Re_base.flatten()

# Combine into a single array with the shape (num, 3)
combined_grid = np.stack((Ma_flat, AoA_flat, Re_base_flat), axis=-1)

latent_geom_expand=[]
for i in range(combined_grid.shape[0]):
    latent_geom_expand.append(np.concatenate([latent_geom[0], combined_grid[i]]))
latent_geom_expand=np.array(latent_geom_expand)

airfoils, c81_mean, c81_std, c81_std2 = airfoil_brain.evaluate(latent_geom_expand)
c81_mean=c81_mean.reshape(Ma_grid.shape[0],AoA_grid.shape[0],Re_base_grid.shape[0],3)
c81_std=c81_std.reshape(Ma_grid.shape[0],AoA_grid.shape[0],Re_base_grid.shape[0],3)
c81_std2[0]=c81_std2[0].reshape(Ma_grid.shape[0],AoA_grid.shape[0],Re_base_grid.shape[0],3)
c81_std2[1]=c81_std2[1].reshape(Ma_grid.shape[0],AoA_grid.shape[0],Re_base_grid.shape[0],3)

print(c81_mean.shape)
from tools.visualization import plot_c81_results
plot_c81_results(c81_mean=c81_mean[:,:,0], c81_std=c81_std[:,:,0], geom=airfoils[0], Ma_grid=Ma_grid, AoA_grid=AoA_grid, \
    levels_list=[np.linspace(-1.5,2.0,21), np.linspace(0,0.8,21), np.linspace(-0.35, 0.35,21)])

In [ ]:
# find the latent variables of target airfoil
target_airfoil_geom=np.loadtxt('representative_airfoil_validation/rae2822.dat', skiprows=1)[:,:2]

plt.plot(target_airfoil_geom[:201,0],target_airfoil_geom[:201,1], 'b-', label='target upper')
plt.plot(target_airfoil_geom[201:,0],target_airfoil_geom[201:,1], 'r--', label='target lower')
plt.legend(frameon=False)
plt.show()

# split the target airfoil into upper/lower surface (direction should be LE to TE)
target_upper=np.flip(target_airfoil_geom[:201],axis=0) 
target_lower=target_airfoil_geom[200:]

fitted_latent, fitted_airfoil, results = airfoil_brain.find_target_airfoil(target_lower, target_upper)
fitted_latent=np.expand_dims(fitted_latent,axis=0)

# flow condition
Ma=0.7 
AoA=2.3
Re_base=6e6

flow_con=np.array([[Ma, AoA, Re_base]])

latent=np.concatenate([fitted_latent, flow_con],axis=1) # batch x features formatting
print(latent.shape)

airfoils, c81_mean, c81_std, c81_std_decompose = airfoil_brain.evaluate(latent, norm=True)
c81_mean[:,1]=10**c81_mean[:,1] # log transformation of drag coefficient


In [ ]:
# geom_save (default is ./geom_save folder)

latent_random_sample=np.random.uniform(0.,1.,size=(10,10))
airfoil_brain.geom_save(latent_random_sample, norm=True)
